Topic Analysis

In [1]:
from google.colab import drive
import os

  # gdrive_path='/content/gdrive/MyDrive/Bertopic/shared_work/'

  # # This will mount your google drive under 'MyDrive'
# drive.mount('/content/gdrive', force_remount=True)
# # In order to access the files in this notebook we have to navigate to the correct folder
# os.chdir(gdrive_path)
# dataset_path = ''
# # Check manually if all files are present
# print(sorted(os.listdir()))

# To run from the common drive:
dataset_path = '/content/drive/MyDrive/Praktikum - NLP Applications/Models/bertopic_concatenated_chunks_stratified'
drive.mount('/content/drive', force_remount=True)


MessageError: Error: credential propagation was unsuccessful

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as ani

In [ ]:
import os
current_path = os.getcwd()
print("Current working directory:", current_path)


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.feature_extraction.text import CountVectorizer
# from bertopic import BERTopic
# from umap import UMAP

# set path of the file
csv_file_path = '/results/topics_for_bias_analysis.csv'
df = pd.read_csv(dataset_path + csv_file_path)

In [ ]:
df.head()

In [ ]:
dataset = df[['title', 'artist', 'cleaned_lyrics', 'views', 'genre', 'year', 'gender', 'topic', 'topic_prob', 'topic_label']]

In [ ]:
dataset.describe()

In [ ]:
# Converting year to decade
dataset['year'].unique()

In [ ]:
dataset['year'].max()

In [ ]:
# Number of songs with year < 1950 - a few of these are from the 18th and 19th centuries but there are also some mistagged songs

print(len(dataset[dataset['year'] < 1950])) # => 2856, only 0.5% of the data so these are ignored
dataset = dataset[dataset['year'] >= 1950]

def get_decade(year):
  if year >= 2000 and year < 2010:
    decade = '2000s'
  else:
    decade = str(year//100) + str(year%100)[0] + '0s'
  return decade

dataset['decade'] = dataset['year'].apply(get_decade)

In [ ]:
# Number of outliers

len(df[df['topic'] == -1])

In [ ]:
dataset = dataset[dataset['topic'] != -1]
dataset = dataset[dataset['genre'] != 'misc']

In [ ]:
dataset.head()

In [ ]:
# What are the most common topics in each genre?
genre_counts_df = dataset.groupby(['genre','topic_label']).size().sort_values().groupby(level=0).tail(5).to_frame('count').reset_index()
genre_counts_df = genre_counts_df.sort_values(['genre', 'count'], ascending=[True, False])

In [ ]:
genre_counts_df

In [ ]:
# Most popular topics per decade

decade_counts_df = dataset.groupby(['decade','topic_label']).size().sort_values().groupby(level=0).tail(5).to_frame('count').reset_index()
decade_counts_df = decade_counts_df.sort_values(['decade', 'count'], ascending=[True, False])

In [ ]:
decade_counts_df

Bar Chart Race

In [ ]:
decade_top10 = dataset.groupby(['decade','topic_label']).size().sort_values().groupby(level=0).tail(10).to_frame('count').reset_index()
decade_top10 = decade_top10.sort_values(['decade', 'count'], ascending=[True, False])
decade_top10 = decade_top10.set_index('decade')
decade_top10 = decade_top10.pivot_table(values='count', index=decade_top10.index, columns='topic_label', aggfunc='first')

In [ ]:
decade_top10 = decade_top10.fillna(0)

In [ ]:
decade_top10

In [ ]:
dec_50s = decade_top10.loc['1950s']
dec_50s

In [ ]:
fig, ax = plt.subplots(figsize=(6, 10), dpi=144)
colors = plt.cm.Dark2(range(8))
y = dec_50s.index
width = dec_50s.values
ax.barh(y=y, width=width, color=colors);

In [ ]:
def nice_axes(ax):
    ax.set_facecolor('.8')
    ax.tick_params(labelsize=8, length=0)
    ax.grid(True, axis='x', color='white')
    ax.set_axisbelow(True)
    [spine.set_visible(False) for spine in ax.spines.values()]

nice_axes(ax)
fig

In [ ]:
fig, ax_array = plt.subplots(nrows=1, ncols=2, figsize=(20, 10),
                             dpi=144, tight_layout=True)
dates = ['1950s', '1960s']
for ax, date in zip(ax_array, dates):
    s = decade_top10.loc[date].sort_values()
    y = decade_top10.loc[date].rank(method='first')
    ax.barh(y=y, width=s.values, color=colors, tick_label=s.index)
    ax.set_title(date, fontsize='smaller')
    nice_axes(ax)

In [ ]:
fig, ax_array = plt.subplots(nrows=1, ncols=2, figsize=(20, 10),
                             dpi=144, tight_layout=True)
dates = ['1970s', '1980s']
for ax, date in zip(ax_array, dates):
    s = decade_top10.loc[date].sort_values()
    y = decade_top10.loc[date].rank(method='first')
    ax.barh(y=y, width=s.values, color=colors, tick_label=s.index)
    ax.set_title(date, fontsize='smaller')
    nice_axes(ax)

In [ ]:
fig, ax_array = plt.subplots(nrows=1, ncols=4, figsize=(20, 10),
                             dpi=144, tight_layout=True)
dates = ['1990s','2000s','2010s','2020s']
for ax, date in zip(ax_array, dates):
    s = decade_top10.loc[date].sort_values()
    y = decade_top10.loc[date].rank(method='first')
    ax.barh(y=y, width=s.values, color=colors, tick_label=s.index)
    ax.set_title(date, fontsize='smaller')
    nice_axes(ax)

In [ ]:
# Top 5 topics per genre and decade

genre_decade_df = dataset.groupby(['genre','decade','topic_label']).size().sort_values().groupby(level=[0,1]).tail(5).to_frame('count').reset_index()
genre_decade_df = genre_decade_df.sort_values(['genre', 'decade', 'count'], ascending=[True, True, False])

In [ ]:
genre_decade_df

In [ ]:
genre_decade_df

In [ ]:
# Percentage of rap songs tagged as 'nigga_nigga_bitch'

rap_songs = dataset[dataset['genre'] == 'rap']
topic_rap = rap_songs[rap_songs['topic_label'] == 'nigga_niggas_bitch']

len(topic_rap)/len(rap_songs) * 100

In [ ]:
# country_songs = dataset[dataset['genre'] == 'rock']
# country_songs.groupby('gender').mean()
# country_songs
dataset.groupby('genre').mean()

In [ ]:
country_songs = dataset[dataset['decade'] == '1990s']
# country_songs.count()
tears_heart_wish_topic = country_songs[country_songs['topic_label'] == 'military time_military_time military']
# male_2 = country_songs[country_songs['gender'] == 'Male']
# male_2
# female_2 = country_songs[country_songs['gender'] == 'Female']
# print("Male: " + str(male_2.count().title) + " / " + 'Female: ' + str(female_2.count().title))
# print('Female: ' + str(female_2.count().title))
# tears_heart_wish_topic
# country_songs.groupby(['genre','gender']).count()
tears_heart_wish_topic.groupby('genre').count()

In [ ]:
# Decade analysis
  # 1950s

    # 7112 songs - Male: 4844 / Female: 2268

    # country	1457 -> 1304 Male / 153 Female
    # pop	4711 -> 2741 Male / 1970 Female
    # rap	4 -> 2 Male / 2 Female
    # rb	388 -> 299 Male / 89 Female
    # rock	552 -> 498 Male / 54 Female

    # 1. 4 wonderful_sweeter years_sweeter Count: 238 -> country 33, pop 195, rb 4, rock 6
    # 2. 3 morning heartache_good morning_twilight Count: 134 -> country 6, pop	124, rb	1, rock 3
    # 3. 2 moon_moonlight_luna Count: 133 -> country 13, pop 117, rb	3
    # 4. 1 blues_worried_blues blues Count: 131 -> country 27, pop 77, rb	13, rock 14
    # 5. 0 love_make love_baby Count: 99 -> country	10, pop	53, rb 15, rock	21

  # 1960s

    # 22576 songs -> Male: 16131 / Female: 6445

    # country	5099 -> 4208 Male / 891 Female
    # pop	13258  -> 8599 Male / 4659 Female
    # rap	14 -> 11 Male / 3 Female
    # rb	2036  -> 1382 Male / 654 Female
    # rock	2169  -> 1931 Male / 238 Female

    # 1. 19 wonderful_sweeter years_sweeter Count: 521 -> country	75, pop	403, rb	17, rock	26
    # 2. 14	tears_heart_wish Count: 468 -> country	133, pop	246, rb	57, rock	32
    # 3. 7 love_make love_baby Count: 329 -> country	18, pop	168, rb	97, rock	46
    # 4. 6 beautiful beautiful_na_beautiful Count: 324 -> country	97, pop	187, rb	19, rock	21
    # 5. 5 goodbye_farewell party_darling Count: 250 -> country	72, pop	146, rb	16, rock	16

  # 1970s

    # 29746 songs -> Male: 22785 / Female: 6961

    # country	5294 -> 4043 Male / 1251 Female
    # pop	16297 -> 11870 Male / 4427 Female
    # rap	59 -> 54 Male / 5 Female
    # rb	2940 -> 2098 Male / 842 Female
    # rock	5156 -> 4720 Male / 436 Female

    # 1. 17 tears_heart_wish Count: 507 -> country	121, pop	243, rap	1, rb	83, rock	59
    # 2. 12	dance_funky_dance dance Count: 414 -> country	15, pop	187, rap	1, rb	130, rock	81
    # 3. 11	beautiful beautiful_na_beautiful Count: 413 -> country	112, pop	235, rb	22, rock	44
    # 4. 9 love_make love_baby Count: 394 -> country	17, pop	203, rap	2, rb	123, rock	49
    # 5. 8 shes_shell_borderlines Count: 359 -> country	139, pop	161, rb	9, rock	50

  # 1980s

    # 32332 songs -> Male: 23504 / Female: 8828

    # country	3328 -> 2428 Male / 900 Female
    # pop	20646 -> 14257 Male / 6389 Female
    # rap	875 -> 803 Male / 72 Female
    # rb	2642 -> 1629 Male / 1013 Female
    # rock	4841 -> 4387 Male / 454 Female

    # 1. 22 tears_heart_wish Count: 661 -> country	85, pop	439, rap	1, rb	70, rock	66
    # 2. 16 jesus_praise_lord Count: 501 -> country	36, pop	385, rap	1, rb	47, rock	32
    # 3. 15 heartache_bah_bah bah Count: 480 -> country	70, pop	304, rb	38, rock	68
    # 4. 13 dance_funky_dance dance Count: 463 -> country	14, pop	264, rap	54, rb	82, rock	49
    # 5. 10 love_make love_baby Count:410 -> country	17, pop	198, rap	7, rb	136, rock	52

  # 1990s

    # 62042 songs -> Male: 43933 / Female: 18109

    # country	4807 -> 3428 Male / 1379 Female
    # pop	38393 -> 24862 Male / 13531 Female
    # rap	8464 -> 7914 Male / 550 Female
    # rb	3644 -> 1982 Male / 1662 Female
    # rock	6734 -> 5747 Male / 987 Female

    # 1. 36 nigga_niggas_bitch Count: 4048 -> country	10, pop	185, rap	3797, rb	19, rock	37
    # 2. 27 jesus_praise_lord Count: 1049 -> country	59, pop	854, rap	7, rb	78, rock	51
    # 3. 26 tears_heart_wish Count: 945 -> country	129, pop	660, rap	5, rb	94, rock	57
    # 4. 24 body_girl_baby Count: 726 -> country	15, pop	312, rap	149, rb	190, rock	60
    # 5. 23 military time_military_time military Count: 702 -> country	2, pop	53, rap	629, rb	1, rock	17

  # 2000s

    # 121158 songs -> Male: 83457 / Female: 37701

    # country	5676 -> 4178 Male / 1498 Female
    # pop	73925 -> 43508 Male / 30417 Female
    # rap	26050 -> 24789 Male / 1261 Female
    # rb	5350 -> 2713 Male / 2637 Female
    # rock	10157 -> 8269 Male / 1888 Female

    # 1. 38 nigga_niggas_bitch Count: 11338 -> country	6, pop	841, rap	10385, rb	84, rock	22
    # 2. 32 body_girl_baby Count: 2272 -> country	30, pop	1136, rap	594, rb	444, rock	68
    # 3. 30 jesus_praise_lord Count: 1846 -> country	99, pop	1485, rap	35, rb	61, rock	166
    # 4. 29 ayy ayy_change_long sentiment Count: 1813 -> country	59, pop	1362, rap	104, rb	61, rock	227
    # 5. 28 tears_heart_wish Count: 1646 -> country	77, pop	1267, rap	36, rb	173, rock	93

    # RB artists male and female same amount.
    # female artists really less compared with male artists in rap sector

  # 2010s

    # 214002 songs -> Male: 157100 / Female: 56902

    # country	9948 -> 7139 Male / 2809 Female
    # pop	123272 -> 79226 Male / 44047 Female
    # rap	50787 -> 48323 Male / 2465 Female
    # rb	10425 -> 6668 Male / 3757 Female
    # rock	19570 -> 15746 Male / 3824 Female

    # 1. 39 nigga_niggas_bitch Count: 21935 -> country	14, pop	2572, rap	19036, rb	272, rock	40
    # 2. 37 body_girl_baby Count: 4608 -> country	140, pop	2429, rap	978, rb	920, rock	141
    # 3. 35 em youre_worth em_ahahah Count: 3009 -> country	51, pop	1019, rap	1643, rb	184, rock	112
    # 4. 34 heartache_bah_bah bah Count: 3007 -> country	255, pop	2070, rap	196, rb	256, rock	230
    # 5. 33 ayy ayy_change_long sentiment Count: 2884 -> country	128, pop	1865, rap	337, rb	130, rock	424

    # Lots of rap songs by male artists compared with female artists (48323/2465)

  # 2020s

    # 29932 songs -> Male: 20167 / Female: 9765

    # country	2233 -> 1437 Male / 796 Female
    # pop	14056 -> 7031 Male / 7025 Female
    # rap	7037 -> 6679 Male / 359 Female
    # rb	2549 -> 1615 Male / 934 Female
    # rock	4057 -> 3406 Male / 651 Female

    # 1. 31 nigga_niggas_bitch Count: 2072 -> pop	51, rap	1974, rb 36, rock	11
    # 2. 25 body_girl_baby Count: 759 -> country 22, pop 327, rap	158, rb	225, rock	27
    # 3. 21 heartache_bah_bah bah Count: 611 -> country	74, pop	350, rap	52, rb	88, rock	47
    # 4. 20 ayy ayy_change_long sentiment Count: 557 -> country	22, pop	325, rap	64, rb	30, rock	116
    # 5. 18 em youre_worth em_ahahah Count: 509 -> country 16, pop 107, rap	305, rb	50, rock 31

    # Here we can see in 2020s male and female pop singers are in same amount.
    # Even though the most of the songs in 2020s are in pop genre, the first topic is rap related topic. This can be because pop songs share diverse topic area or
    # rap minimum topic area meaning most of the rappers always use vulgar words and racism-vulgarism. 2nd, 3rd and 4th topic are all
    # pop related while 5th topic again rap related

    # During the years we can see trend of country music decreasing and rap increasing. Despite the increase of rap from 1950 to 2010, for some reason we see less rap songs
    # in 2020s. This can be dataset shortage problem.

    # nigga_niggas_bitch has been top topic from 1990-2020s
    # tears_heart_wish has been top topic from 1970-1980s
    # wonderful_sweeter years_sweeter has been top topic from 1950-1960s

In [ ]:
# We start our analysis based on Genre
# Country
  # All in all we have 37842 songs in this genre where 28165 (74%) belongs to Male singer and 9677 (26%) to Female singer.
  # Male singers have 56120166 views in total while Female singers have 36082149.
  # But in average this number decreases to 3728 by female singers and 1992 views for male singers.
  # Most of the songs by female singers has released in 1997, while for males in 1992.
  # Top 5 topics:
  # 1. 10 tears_heart_wish Count: 779
      # Male: 485 / Female: 294
  # 2. 9 heartache_bah_bah bah	Count: 686
      # Male: 472 / Female: 214
  # 3. 2 shes_shell_borderlines Count: 619
      # Male: 564 / Female: 55
  # 4. 1 country_hank_thats right Count: 618
      # Male: 568 / Female: 50
  # 5. 0 country_mud_redneck Count: 560
      # Male: 513 / Female: 47

  # As it is seen from the numbers we can see that singers who has songs in country genre are mostly males by showing 74% of all artists.
  # This also results high view numbers of the songs when we sum them. But when we take into account average views of males and females separately
  # we see that Female singers have more views. The top topic for country genre was 'tears_heart_wish' where again male singers were populated.
  # This trend also continues in other topics.

# Pop
  # All in all we have 304560 songs in this genre where 192095 (63%) belongs to Male singer and 112465 (37%) to Female singer.
  # Male singers have 611384071 views in total while Female singers have 588378362.
  # But in average this number decreases to 5232 by Female singers and 3183 views for male singers.
  # Most of the songs by female singers has released in 2004, while for males in 2002.
  # Top 5 topics:
  # 1. 23	jesus_praise_lord	Count: 5400
      # Male: 3487 / Female: 1913
  # 2. 22 tears_heart_wish Count: 5368
      # Male: 2754 / Female: 2614
  # 3. 21 body_girl_baby Count: 4529
      # Male: 2758 / Female: 1771
  # 4. 20 heartache_bah_bah bah Count: 4451
      # Male: 2460 / Female: 1991
  # 5. 19 ayy ayy_change_long sentiment Count: 4424
      # Male: 2583 / Female: 1841

  # Pop songs also show almost same trend with country songs. 63% of all songs have male as singers. But female singer songs have 588 million
  # views which is closer to 611 million views by male singers. This would mean that even though Female singers are less in this genre,
  # but they are talented and famous so they don't have big gap in this view factor.
  # 2nd top topic tears_heart_wish show almost same distribution between males and females while others are all male populated topics.

# Rap
  # All in all we have 93292 songs in this genre where 88575 (95%) belongs to Male singer and 4717 (5%) to Female singer.
  # Male singers have 2377699546 views in total while Female singers have 104205246. (23 times less)
  # But in average this number decreases to 22091 by female singers and 26844 views for male singers.
  # Most of the songs by female singers has released in 2009, while for males in 2010.
  # Top 5 topics:
  # 1. 24	nigga_niggas_bitch Count: 35344
      # Male: 34090 / Female: 1253
  # 2. 18 military time_military_time military Count: 4234
      # Male: 4169 / Female: 65
  # 3. 17 em youre_worth em_ahahah Count: 2932
      # Male: 2638 / Female: 294
  # 4. 16 tut_tut tut_aes Count: 2639
      # Male: 2555 / Female: 84
  # 5. 15 ra_viral_body body Count: 2550
      # Male: 2362 / Female: 188

  # 95% of the rap songs belong to males. This number is really high that it also creates 23 times gap between the view size of each gender's songs view.
  # In average males also has high views having 26844 views while females have 22091 views.
  # Top topic for rap songs is nigga_niggas_bitch which has 96% of males as singers. They are more prone to sing in this topic.
  # 2nd top topic - Military topic has less female songs considered to other top topics, this can be reason in the countries where males are more streotyped to go military
  # or having compulsory army service in countries. We see also big gap between first and second top topic (35344/4234).

# RB
  # All in all we have 29974 songs in this genre where 18386 (61%) belongs to Male singer and 11588 (39%) to Female singer.
  # Male singers have 349753881 views in total while Female singers have 132648855. (2.64 times less)
  # But in average this number decreases to 11447 by female singers and 19023 views for male singers.
  # Most of the songs by female singers has released in 2001, while for males in 2000.
  # Top 5 topics:
  # 1. 14	body_girl_baby Count: 1924
      # Male: 1389 / Female: 535
  # 2. 13 love_make love_baby	Count: 1159
      # Male: 703 / Female: 456
  # 3. 11 tears_heart_wish Count: 817
      # Male: 413 / Female: 404
  # 4. 7 dance_funky_dance dance Count: 666
      # Male: 442 / Female: 224
  # 5. 3 heartache_bah_bah bah Count: 621
      # Male: 331 / Female: 290

  # 61% of the songs have male singers. Around other genres Females has more percentages for RB genre,
  # but if we take numbers Pop genre has more females (112465 with 37%). Even though female singers have more views in RB than Pop, they are still 3 times less
  # than male singers in RB. Their small amount couldnt overcome this issue like Pop. Male singers have more views in average.
  # Top topic body_girl_baby shows 72% majority in male singers. But again as in Pop we see 3rd top topic tears_heart_wish have almost same distribution with males and females

# Rock
  # All in all we have 53236 songs in this genre where 44704 (84%) belongs to Male singer and 8532 (16%) to Female singer.
  # Male singers have 216055743 views in total while Female singers have 43297110. (5 times less)
  # But in average this number decreases to 5075 by female singers and 4833 views for male singers.
  # Most of the songs by female singers has released in 2005, while for males in 2001.
  # Top 5 topics:
  # 1. 12	ayy ayy_change_long sentiment	Count: 960
      # Male: 788 / Female: 172
  # 2. 8 shadow_light_moon Count: 683
      # Male: 577 / Female: 106
  # 3. 6 death unlike_unlike_whoo woahwoah Count: 656
      # Male: 556 / Female: 100
  # 4. 5 sudden felt_moving sound_felt body Count: 640
      # Male: 534 / Female: 106
  # 5. 4 burn burn_burn_line survive Count: 629
      # Male: 502 / Female: 127

  # Again and again males are populated in this genre as well with 84% of all songs. Male singers also have 5 times more views in total than Females when summed up
  # When it comes to average of the views separately Females have more views, meaning their less amount didn't diminish their average view but they are more played and viewed
  # Top topic 'ayy ayy_change_long sentiment' has 82% of male singers. Around top topics females show less interest in 3rd top topic - 'death unlike_unlike_whoo woahwoah'
  # They are less interested in singing songs related to death, killing or any kind of violance.

# Repeated topics around genres:
  # 1. tears_heart_wish -> Country, Pop, RB
  # 2. heartache_bah_bah bah -> Country, Pop, RB
  # 3. body_girl_baby -> Pop, RB
  # 4. ayy ayy_change_long sentiment -> Pop, Rock

# Pop songs topic are more general that it can be repeated in other genres. Pop and RB share 3 same topics. This again brings us to a question what is the difference between
# this 2 genres. Usually Pop songs are more upbeat while RB is more chill. Sometimes artists experiment by combining both styles into their own unique sound
# which can make identifying songs even more challenging.

In [ ]:
military_songs = dataset[dataset['topic_label'] == 'military time_military_time military']
# military_songs = military_songs[military_songs['genre'] == 'rap']
military_songs.groupby(['genre','gender']).count()
# military_songs.groupby('gender').count()

In [ ]:
military_songs = dataset[dataset['topic_label'] == 'military time_military_time military']
military_songs = military_songs[military_songs['title'] == 'Massive Attack'] # Nicki Minaj
list(military_songs['cleaned_lyrics'])
# Tom-Toms -> A acronym for a Thompson SubMachine Gun

In [ ]:
military_songs = dataset[dataset['topic_label'] == 'military time_military_time military']
military_songs = military_songs[military_songs['title'] == 'I Cant Feel My Face'] # Lil Wayne
list(military_songs['cleaned_lyrics'])

# "Shoot a nigga on his porch and make him fall in his kitchen" - While violent, this lyric exposes the harsh realities of certain urban environments.

# 'Bout to call Bin Laden up and order some missiles

# When the gun goes "Pow," I be at the finish

In [ ]:
military_songs = dataset[dataset['topic_label'] == 'military time_military_time military']
military_songs = military_songs[military_songs['title'] == '​t h e . c l i m b . b a c k'] # J. Cole
list(military_songs['cleaned_lyrics'])

# And paint the town red for my nigga found dead too soon

# More death than World War II caused\nAround these parts we pour the brown just to drown these thoughts

# Need a number for my niggas to call\nWhenever there\'s a urge to get triggers involved'

In [ ]:
tut_tut_songs = dataset[dataset['topic_label'] == 'tut_tut tut_aes']
tut_tut_songs = tut_tut_songs[tut_tut_songs['views'] > 1000000]
tut_tut_songs

In [ ]:
tut_tut_songs = dataset[dataset['topic_label'] == 'tut_tut tut_aes']

# tut_tut_songs = tut_tut_songs[tut_tut_songs['title'] == 'The Monster']

# The song "The Monster" by Eminem (ft. Rihanna) is about coming to terms with one's inner struggles, and being able to accept the darker aspects of oneself.
# The song speaks to being "friends with the monster" within, rather than trying to deny or push away one's inner chaos. The lyrics talk about
# living with the pressures of fame and success, while also trying to cope with their own inner demons and self-doubt. The chorus speaks to the
# idea of trying to save oneself, but also realizing that pushing away the voices inside one's head will only make them stronger. Ultimately,
# the song is about acknowledging one's inner darkness and accepting it, in order to ultimately be able to live with and conquer it.
# https://www.songtell.com/eminem-ft-rihanna/the-monster

# tut_tut_songs = tut_tut_songs[tut_tut_songs['title'] == 'The World Is Yours']

# This song deals with the struggles Nas faced growing up in New York City, or as he calls it in this song,
# "The Rotten Apple." Some of the challenges he raps about overcoming include poverty, violence and racist cops.
# https://www.songfacts.com/facts/nas/the-world-is-yours

tut_tut_songs = tut_tut_songs[tut_tut_songs['title'] == 'Rigamortis']

list(tut_tut_songs['cleaned_lyrics'])

# Got me breathing with dragons
# This line is an interesting play on the phrase "I spit fire". Although rappers have compared their flow to lava (Ice Cube) or to bombs (Shaquille O'Neal),
# no one has ever compared their flow to that of a dragon, who literally spits fire.

# I'm on the toilet when I rhyme, if you the shit, then I decline
# This line demonstrates that Kendrick does not follow society, as people

# So this topic is more about rappers development through years, personal growth, how hard it was to come to the peek.

In [ ]:
tears_heart_wish_songs = dataset[dataset['topic_label'] == 'tears_heart_wish']
tears_heart_wish_songs = tears_heart_wish_songs[tears_heart_wish_songs['views'] > 1000000]
tears_heart_wish_songs.groupby(['genre','gender']).count()

In [ ]:
tears_heart_wish_songs = dataset[dataset['topic_label'] == 'tears_heart_wish']
tears_heart_wish_songs = tears_heart_wish_songs[tears_heart_wish_songs['gender'] == 'Female']
tears_heart_wish_songs = tears_heart_wish_songs[tears_heart_wish_songs['views'] > 1000000]
tears_heart_wish_songs

In [ ]:
tears_heart_wish_songs = dataset[dataset['topic_label'] == 'tears_heart_wish']
tears_heart_wish_songs = tears_heart_wish_songs[tears_heart_wish_songs['gender'] == 'Male']
tears_heart_wish_songs = tears_heart_wish_songs[tears_heart_wish_songs['views'] > 1000000]
tears_heart_wish_songs

In [ ]:
save_your_tears_song = dataset[dataset['title'] == 'Save Your Tears']

list(save_your_tears_song['cleaned_lyrics'])

# "Save Your Tears" captures The Weeknd's longing for Hadid after seeing her out and his regrets over breaking her heart.
# The song opens with the lyrics "I saw you dancing in a crowded room/You look so happy when I'm not with you/But then you saw me,
# caught you by surprise/A single teardrop falling from your eye." The singer hints at a rift between him and a partner who looks happier
# without him, only to become emotional when they cross paths by chance. Art imitates reality here: As TMZ reported, Hadid apparently avoided an
# awkward encounter with The Weeknd at a nightclub after their breakup in 2019.

# https://www.elle.com/culture/music/a31805976/the-weeknd-save-your-tears-lyrics-bella-hadid-selena-gomez/

In [ ]:
someone_like_you_song = dataset[dataset['title'] == 'Someone Like You']
someone_like_you_song = someone_like_you_song[someone_like_you_song['artist'] == 'Adele']

list(someone_like_you_song['cleaned_lyrics'])

# It begins with - "I heard that you're settled down, That you found a girl and you're married now," immediately set the tone for a sense of loss.
# Artist reflects on hearing news about her former lover's new life, which reflexts with her current emotional state.

# The chorus lines, "Never mind, I'll find someone like you, I wish nothing but the best for you," demonstrate a bittersweet acceptance.
# While there's an evident longing for the past, Adele also expresses a genuine desire for her former lover's happiness, even if it's not with her.

# There is repeated line: "Sometimes it lasts in love, but sometimes it hurts instead," captures the dual nature of love as both enduring and painful.
# Meaning is that while love can be lasting and beautiful, it can also lead to deep emotional pain when it doesn't work out.

In [ ]:
genre_counts_df = dataset.groupby(['genre','decade']).size()
genre_counts_df

df = genre_counts_df.reset_index()
df.columns = ['genre', 'decade', 'count']

# Pivot the DataFrame
pivot_df = df.pivot(index='genre', columns='decade', values='count')

# Plotting
plt.figure(figsize=(12, 8))
pivot_df.plot(kind='bar', stacked=True, colormap='viridis', ax=plt.gca())

plt.title('Song Counts by Genre and Decade')
plt.xlabel('Genre')
plt.ylabel('Song Count')
plt.xticks(rotation=45)
plt.legend(title='Decade', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

plt.show()

In [ ]:
df = genre_counts_df.reset_index()
df.columns = ['genre', 'decade', 'count']

# Pivot the DataFrame
pivot_df = df.pivot(index='genre', columns='decade', values='count')

# Plotting
plt.figure(figsize=(15, 10))

# Plot each genre as a separate line
for genre in pivot_df.index:
    plt.plot(pivot_df.columns, pivot_df.loc[genre, :], marker='o', label=genre)

plt.title('Song Count Trends by Genre and Decade')
plt.xlabel('Decade')
plt.ylabel('Song Count')
plt.xticks(rotation=45)
plt.legend(title='Genre')
plt.grid(True)
plt.tight_layout()

plt.show()

In [ ]:
topic_decade_df = dataset.groupby(['decade','topic_label']).size().sort_values().groupby(level=[0,1]).tail(5).to_frame('count').reset_index()
topic_decade_df = topic_decade_df.sort_values(['decade', 'count'], ascending=[True, False])
topic_decade_df

In [ ]:
topic_decade_df = topic_decade_df[topic_decade_df['count'] > 2000]

# Pivot the DataFrame to get decades as columns and topics as rows
pivot_df = topic_decade_df.pivot(index='topic_label', columns='decade', values='count').fillna(0)

# Plotting
plt.figure(figsize=(15, 10))

# Plot each topic as a separate line
for topic in pivot_df.index:
    plt.plot(pivot_df.columns, pivot_df.loc[topic, :], marker='o', label=topic)

plt.title('Topic Development Through Decades')
plt.xlabel('Decade')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.legend(title='Topic Label', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=13)
plt.grid(True)
plt.tight_layout()

plt.show()

In [ ]:
# Count the number of male and female artists in rap
import seaborn as sns
gender_counts = dataset.groupby('genre')['gender'].value_counts().reset_index(name='count')

# # Plotting the pie chart
# plt.figure(figsize=(8, 6))
# plt.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=140)
# plt.title('Gender Distribution in Rap Genre Songs')
# plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle

# plt.show()

sns.barplot(gender_counts, x = 'genre', y = 'count', hue='gender').set_title("Artist Gender Distribution for each Genre")

In [ ]:
gender_counts

In [ ]:
# Pivot the DataFrame to reformat it
new_gender_counts = gender_counts.pivot(index='genre', columns='gender', values='count').reset_index()

# Display the transformed DataFrame
# new_gender_counts = new_gender_counts.drop('gender', axis=1)
new_gender_counts

In [ ]:
new_gender_counts['Female_Proportion'] = new_gender_counts['Female'] / (new_gender_counts['Female'] + new_gender_counts['Male'])
new_gender_counts['Male_Proportion'] = new_gender_counts['Male'] / (new_gender_counts['Female'] + new_gender_counts['Male'])

# Keeping only the necessary columns
new_gender_counts = new_gender_counts[['genre', 'Female_Proportion', 'Male_Proportion']]

new_gender_counts

In [ ]:
new_gender_counts.Female_Proportion.dtype

In [ ]:
new_gender_countss = gender_counts.pivot(index='genre', columns='gender', values='count').reset_index()
new_gender_countss

In [ ]:
new_gender_countss.Female.dtype

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

gender_counts_prop = pd.crosstab(index=new_gender_counts['genre'],
                             columns=new_gender_counts['Female'],
                             normalize="index")
gender_counts_prop

In [ ]:
new_gender_counts.plot(kind='bar',
                    stacked=True,
                    colormap='tab10',
                    figsize=(10, 6))

plt.legend(loc="upper left", ncol=2)
plt.xlabel("genre")
plt.ylabel("Artist Gender")
plt.xticks(ticks = [0,1,2,3,4], labels = new_gender_counts['genre'])
# plt.show()


for n, x in enumerate([*new_gender_countss.index.values]):
    for proportion in new_gender_counts.loc[x]:
        print(proportion)
        plt.text(x=n,
                 y=proportion,
                 s=f'{np.round(proportion * 100, 1)}%',
                 color="black",
                 fontsize=12,
                 fontweight="bold")

plt.show()

In [ ]:
rap_df = dataset[dataset['genre'] == 'rap']
rap_df = rap_df.drop(columns=['title', 'artist', 'cleaned_lyrics', 'views', 'year', 'topic_prob', 'topic', 'decade'])
rap_df

# Count the number of male and female artists in rap
gender_counts = rap_df['gender'].value_counts()

# Plotting the pie chart
plt.figure(figsize=(8, 6))
plt.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Gender Distribution in Rap Genre Songs')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle

plt.show()

In [ ]:
data_2 = dataset[['genre', 'gender']]
data_2

In [ ]:
cross_tab_prop = pd.crosstab(index=data_2['genre'],
                             columns=data_2['gender'],
                             normalize="index")
cross_tab_prop

In [ ]:
cross_tab = pd.crosstab(index=data_2['genre'],
                        columns=data_2['gender'])
cross_tab

In [ ]:
cross_tab_prop.plot(kind='bar',
                    stacked=True,
                    colormap='tab10',
                    figsize=(10, 6))

plt.legend(loc="upper left", ncol=2)
plt.xlabel("Genre")
plt.ylabel("Artist Gender")


for n, x in enumerate([*cross_tab.index.values]):
    for proportion in cross_tab_prop.loc[x]:

        plt.text(x=n,
                 y=proportion,
                 s=f'{np.round(proportion * 100, 1)}%',
                 color="black",
                 fontsize=12,
                 fontweight="bold")

plt.show()